In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

In [3]:
import pandas as pd

In [76]:
bank_df = pd.read_csv('Churn_Modelling.csv')

In [65]:
bank_df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [ ]:
#RowNumber, CustomerID, Surname is not required for prediction. Hence dropping these cols.

In [77]:
bank_df = bank_df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [78]:
#Labelencoding the string
X1 = pd.get_dummies(bank_df['Geography'], drop_first=False)

In [79]:
X2 = pd.get_dummies(bank_df['Gender'], drop_first=False)

In [80]:
bank_df = pd.concat([bank_df, X1, X2], axis=1, join='inner')

In [81]:
bank_df = bank_df.drop(['Geography','Gender'],axis=1)

In [82]:
#Removing ouliers from the dataset
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(bank_df))

In [83]:
bank_df = bank_df[(z < 3).all(axis=1)]

In [84]:
#Defining the feature set and target variable
X = bank_df.drop('Exited',axis=1)

In [86]:
Y = bank_df['Exited']

In [87]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 5)

In [88]:
x_train.shape

(7839, 13)

In [89]:
# Standardizing the dataset
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(x_train)
X_test = sc_X.transform(x_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [90]:
from keras import Sequential
from keras.layers import Dense

In [91]:
x_train.head(10)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain,Female,Male
168,667,39,2,0.00,2,1,0,40721.24,0,0,1,1,0
3389,420,55,4,91893.32,1,1,0,144870.28,0,0,1,1,0
1791,702,35,8,14262.80,2,1,0,54689.16,0,0,1,1,0
5260,704,35,3,154206.07,2,1,1,40261.49,0,1,0,1,0
4322,508,31,8,72541.48,1,1,0,129803.08,1,0,0,0,1
2146,850,49,5,122486.47,1,0,1,59748.19,1,0,0,0,1
2367,620,39,9,159492.79,1,1,0,80582.34,0,1,0,0,1
7331,797,59,4,129321.44,1,1,1,93624.55,0,0,1,0,1
8116,640,43,9,94752.49,1,1,0,184006.36,0,1,0,1,0
2384,732,40,10,0.00,2,1,0,154189.08,1,0,0,1,0


In [138]:
# In this model, I am taking 2 hidden layer with 4 nodes each. Output layer being binary we
# only one node.
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_uniform', input_dim=13))
#Second  Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_uniform'))
#Third  Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_uniform'))
#Output Layer
classifier.add(Dense(1, activation='relu', kernel_initializer='random_uniform'))

In [139]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [140]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=20)

Epoch 1/20
7839/7839 [==============================] - 2s 317us/step - loss: 0.5028 - acc: 0.8136
Epoch 2/20
7839/7839 [==============================] - 2s 227us/step - loss: 0.3898 - acc: 0.8278
Epoch 3/20
7839/7839 [==============================] - 2s 245us/step - loss: 0.3737 - acc: 0.8385
Epoch 4/20
7839/7839 [==============================] - 2s 246us/step - loss: 0.3673 - acc: 0.8450
Epoch 5/20
7839/7839 [==============================] - 2s 226us/step - loss: 0.3586 - acc: 0.8490
Epoch 6/20
7839/7839 [==============================] - 2s 227us/step - loss: 0.3574 - acc: 0.8491
Epoch 7/20
7839/7839 [==============================] - 2s 220us/step - loss: 0.3530 - acc: 0.8532
Epoch 8/20
7839/7839 [==============================] - 2s 233us/step - loss: 0.3555 - acc: 0.8500
Epoch 9/20
7839/7839 [==============================] - 2s 232us/step - loss: 0.3543 - acc: 0.8533
Epoch 10/20
7839/7839 [==============================] - 2s 230us/step - loss: 0.3516 - acc: 0.8528
Epoch 11/

In [141]:
y_pred1 = classifier.predict(X_test)
print(y_pred1)

[[0.32148793]
 [0.01300352]
 [0.37502375]
 ...
 [0.01249998]
 [0.03356792]
 [0.17936735]]


In [142]:
y_pred1 = (y_pred1 >= 0.5)
print(y_pred1)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [143]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred1)
print(cm1)

[[1505   54]
 [ 227  174]]


In [144]:
print (((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0]), '% of testing data was classified correctly')



85.66326530612245 % of testing data was classified correctly
